In [1]:
import sys
import os
from pathlib import Path

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

GNDTRUTH = Path('../data/raw/COMMON_images_masks/')
RAWIMGS = Path('../data/raw/GROUP_images/')
DATA = (RAWIMGS.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk
from utils import *
from linear_registration import register_linear

In [41]:
mov_names = ['g1_54', 'g1_55', 'g1_56']
fix_name = 'common_42'

fix_im = sitk.ReadImage(GNDTRUTH/(fix_name+'_image.nii.gz'))
mov_ims = [sitk.ReadImage(RAWIMGS/(name+'_image.nii.gz')) for name in mov_names]
fix_msk = sitk.ReadImage(MASKS/(fix_name+'_regmask.nii.gz'))
mov_msks = [sitk.ReadImage(MASKS/(name+'_mask.nii.gz')) for name in mov_names]

# Arrays from images
fix_im_data = sitk.GetArrayFromImage(fix_im)
fix_msk_data = sitk.GetArrayFromImage(fix_msk)
mov_ims_data = [sitk.GetArrayFromImage(img) for img in mov_ims]
mov_msks_data = [sitk.GetArrayFromImage(msk) for msk in mov_msks]


In [43]:
registrations = []
for idx, (mov_im, mov_msk) in enumerate(zip(mov_ims, mov_msks)):
    print(f'---- Image {idx} ----')
    registrations.append(register_linear(fix_im, mov_im, fix_mask=fix_msk, mov_mask=mov_msk, verbose=True))

---- Image 0 ----
Cropping fixed image to ROI...
Cropping moving image to ROI...
Estimating affine transformation...
itk::simple::CompositeTransform
 CompositeTransform (0x2f532680)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 908972
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   AffineTransform (0x2efc8b80)
     RTTI typeinfo:   itk::AffineTransform<double, 3u>
     Reference Count: 1
     Modified Time: 908962
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       1.03928 -0.115763 -0.00401361 
       0.115426 1.07499 0.0505411 
       0.0515849 -0.0160997 0.971747 
     Offset: [-45.5282, 41.5789, -394.181]
     Center: [-70.9805, -134.229, -368.602]
     Translation: [-31.2986, 4.6903, -385.267]
     Inverse: 
       0.950899 0.102378 -0.00139726 
       -0.0996504 0.918786 -0.0481982 
       -0.0521292 0.00978752 1.02835 
     Singular: 0
   TransformsToOpt

In [44]:
mov_ims_reg = [reg['registered_image'] for reg in registrations]
mov_msks_reg = [reg['registered_mask'] for reg in registrations]
lin_tfms = [reg['affine_tfm'] for reg in registrations]

In [45]:
mov_ims_reg_data = [sitk.GetArrayFromImage(im) for im in mov_ims_reg]
mov_msks_reg_data = [sitk.GetArrayFromImage(im) for im in mov_msks_reg]

In [49]:
show_interactive_overlay(fix_im_data, mov_ims_reg_data[2], 'coronal')

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_overlay(fix_arr, mov_arr, slc)>

In [50]:
for idx, name in enumerate(mov_names):
    sitk.WriteImage(mov_ims_reg[idx], DATA/('linear_registrations/with_regmask/'+name+'_linreg_'+fix_name+'_img.nii.gz'))
    sitk.WriteImage(mov_msks_reg[idx], DATA/('linear_registrations/with_regmask/'+name+'_linreg_'+fix_name+'_msk.nii.gz'))
    sitk.WriteTransform(lin_tfms[idx], DATA/('linear_registrations/with_regmask/transforms/'+name+'_linreg_'+fix_name+'.tfm'))

In [9]:
from utils.crop import downsample

mov_dspl = downsample(mov_im_reg, 4)
mov_msk_dspl = downsample(mov_msk_reg, 4)

In [10]:
mov_dspl_data = sitk.GetArrayFromImage(mov_dspl)
mov_msk_dspl_data = sitk.GetArrayFromImage(mov_msk_dspl)
show_interactive(mov_dspl_data, 'coronal', mov_msk_dspl_data)

interactive(children=(IntSlider(value=64, description='slc', max=127), Output()), _dom_classes=('widget-intera…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [12]:
mov_names = ['g1_54', 'g1_55', 'g1_56']
sitk.ReadImage(RAWIMGS/(mov_names[0]+'_image.nii.gz'))

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x7af68cab6b80> >